In [53]:
import optuna
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
import sklearn
from math import sqrt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from optuna.integration import CatBoostPruningCallback
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC

In [54]:
train = pd.read_csv('../data/text_classification.csv')

In [55]:
test = pd.read_csv('../data/text_classification_test.csv')

In [56]:
train['text_len'] = train['text'].apply(lambda x: len(x))
test['text_len'] = test['text'].apply(lambda x: len(x))

In [57]:
targets = ['category']
features2drop = ['text']
filtered_features = [i for i in train.columns if (i not in targets and i not in features2drop)]

In [58]:
class_names = np.unique(train['category'])
train['category'] = train['category'].replace(class_names, np.arange(train['category'].nunique()))

In [59]:
X = train[filtered_features].drop(targets, axis=1, errors='ignore')
y = train['category']

In [60]:
X_test = test[filtered_features]

In [61]:
params_cat = {
             'n_estimators' : 200,
              # 'learning_rate': .03,
              'depth' : 3,
              'verbose': False,
              'use_best_model': True,
              'text_features': [],
              # 'train_dir' : '/home/jovyan/work/catboost',
              'border_count' : 64,
              'l2_leaf_reg' : 1,
              'bagging_temperature' : 2,
              'rsm' : 0.51,
              'loss_function': 'MultiClass',
              'auto_class_weights' : 'Balanced', #try not balanced
              'random_state': 42,
              'use_best_model': False,
              # 'custom_metric' : ['AUC', 'MAP'] # Не работает внутри sklearn.Pipelines
         }



cat_model = CatBoostClassifier(**params_cat)

In [62]:
params_cat_two = {
             'n_estimators' : 200,
              # 'learning_rate': .03,
              'depth' : 3,
              'verbose': False,
              'use_best_model': True,
              'text_features': [],
              # 'train_dir' : '/home/jovyan/work/catboost',
              'border_count' : 64,
              'l2_leaf_reg' : 1,
              'bagging_temperature' : 2,
              'rsm' : 0.51,
              'loss_function': 'MultiClass',
              'auto_class_weights' : 'Balanced', #try not balanced
              'random_state': 52,
              'use_best_model': False,
              # 'custom_metric' : ['AUC', 'MAP'] # Не работает внутри sklearn.Pipelines
         }



cat_model_two = CatBoostClassifier(**params_cat)

In [63]:
skb = SelectKBest(score_func=f_classif, k=1000)
X_new = skb.fit_transform(X, y)

selected_features = skb.get_support(indices=True)

best_features = X.columns[selected_features]

In [64]:
X = X[best_features]
X_test = X_test[best_features]

In [65]:
params_lgbm = {
    "num_leaves": 200,
    "n_estimators": 200,
    # "max_depth": 7,
    "min_child_samples": None,
    "learning_rate": 0.001,
    "min_data_in_leaf": 5,
    "feature_fraction": 0.98,
    # "categorical_feature": cat_cols,
    'reg_alpha' : 3.0,
    'reg_lambda' : 5.0,
}

lgbm_model = LGBMClassifier(**params_lgbm)

In [66]:
params_xgb = {
    "eta": 0.05,
    'n_estimators' : 200,
    "max_depth": 6,
    "subsample": 0.7,
    # "colsample_bytree": 0.95,
    'min_child_weight' : 0.1,
    'gamma': .01,
    'reg_lambda' : 0.1,
    'reg_alpha' : 0.5,
    "objective": "reg:linear",
    "eval_metric": "mae",
    'tree_method' : 'hist', # Supported tree methods for cat fs are `gpu_hist`, `approx`, and `hist`.
    'enable_categorical' : True
    
}

xgb_model = XGBClassifier(**params_xgb)

In [67]:
class Blender:
    def __init__(self, base_models, meta_model):
        self.base_models = base_models
        self.meta_model = meta_model

    def fit(self, X, y):
        n_fold = 3
        base_preds = []
        folds = KFold(n_splits=n_fold)
        
        for train_indices, val_indices in folds.split(X, y):
            X_train, X_val = X.loc[train_indices], X.loc[val_indices]
            y_train, y_val = y[train_indices], y[val_indices]
            models_preds = []
            
            for base_model in self.base_models:
                base_model.fit(X_train, y_train)
                models_preds.append(base_model.predict(X_val))
#                 print(y_val.values.reshape(-1, 1))
                
            base_preds.append(np.column_stack(models_preds))
            
        self.meta_model.fit(np.concatenate(base_preds), y)
            
        return base_preds

    def predict(self, X_test):
        base_preds = []
        
        for base_model in self.base_models:
            base_preds.append(base_model.predict(X_test))
        
        combined_preds = np.column_stack(base_preds)  
        preds = self.meta_model.predict(combined_preds)
        
        return preds

In [68]:
stacker = Blender(
    base_models=[cat_model, cat_model_two],
    meta_model=lgbm_model)

In [69]:
stacker.fit(X, y)

[LightGBM] [Warning] feature_fraction is set=0.98, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.98


[array([[12, 12],
        [ 2,  2],
        [ 4,  4],
        ...,
        [ 7,  7],
        [12, 12],
        [ 7,  7]]),
 array([[12, 12],
        [12, 12],
        [ 4,  4],
        ...,
        [ 9,  9],
        [10, 10],
        [ 6,  6]]),
 array([[ 2,  2],
        [ 8,  8],
        [ 6,  6],
        ...,
        [ 9,  9],
        [10, 10],
        [ 9,  9]])]

In [70]:
test_predict = pd.DataFrame(stacker.predict(X_test), columns=['category'])

In [71]:
test_predict['category'] = test_predict['category'].replace(np.arange(train['category'].nunique()), class_names)

In [72]:
test_predict

,category
0,motosport
1,extreme
2,esport
3,athletics
4,autosport
...,...
2495,boardgames
2496,esport
2497,football
2498,autosport


In [73]:
test_predict.to_csv('../data/nlp_test_predict.csv', index=False)

In [74]:
train

,category,text,ruBert-base_text_feature_0,ruBert-base_text_feature_1,ruBert-base_text_feature_2,ruBert-base_text_feature_3,ruBert-base_text_feature_4,ruBert-base_text_feature_5,ruBert-base_text_feature_6,ruBert-base_text_feature_7,...,labse_text_feature_759,labse_text_feature_760,labse_text_feature_761,labse_text_feature_762,labse_text_feature_763,labse_text_feature_764,labse_text_feature_765,labse_text_feature_766,labse_text_feature_767,text_len
0,5,Ледник Пасторури это цирковой ледник расположе...,0.272156,0.155383,0.060285,0.363159,-0.140391,0.507753,-0.226326,0.431878,...,-0.027475,0.030528,-0.052218,0.042459,-0.012714,-0.055370,-0.012433,-0.016283,-0.006994,139
1,8,Главные участники предстоящего Betokenoid 274 ...,0.439223,0.343683,0.093642,0.245294,0.089770,0.424717,-0.071487,0.185970,...,0.042258,-0.027394,-0.033566,0.016021,-0.022054,-0.040366,0.007392,-0.029070,-0.011284,202
2,5,Ttokenoid Btokenoid – карта с которой можно не...,-0.040338,0.058095,-0.091063,0.296028,-0.137103,0.931456,-0.169060,0.131503,...,0.011548,-0.046034,0.024588,-0.013670,-0.047028,0.009395,-0.000488,-0.060260,0.006563,139
3,1,В Сильверстоуне произошли крупные обновления а...,0.444181,0.218742,0.247859,0.234885,0.006668,0.407703,-0.115768,0.433781,...,-0.043454,-0.000961,-0.012203,-0.047922,-0.054657,-0.053768,0.018481,-0.039148,-0.038874,216
4,5,На протяжении более чем 30 лет Вестсайд являет...,-0.126253,-0.115856,0.131131,0.052595,0.060591,0.420976,0.090776,0.246287,...,-0.008138,-0.013958,-0.038482,-0.002943,-0.035970,-0.010830,-0.005132,-0.047990,-0.005869,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,9,Яркий открытый шлем. ⠀ Внешняя оболочка из выс...,-0.052873,0.115095,0.067304,0.238675,0.042321,0.694844,0.009075,0.298883,...,-0.003430,-0.066267,-0.013982,0.003619,-0.044228,-0.037047,0.005730,-0.076227,-0.048963,818
7496,8,Никто не рождается с идеальным телом. Но если ...,-0.356527,0.173754,0.007272,0.293671,-0.252398,0.243566,0.163168,0.252569,...,-0.004166,-0.044707,-0.062995,-0.009477,0.011715,-0.002244,-0.034123,-0.042895,-0.047827,91
7497,9,Друзья всем привет 33 В феврале продал свою Ho...,0.177330,0.278536,0.013133,0.141096,-0.121019,0.547267,0.000883,0.397169,...,0.037294,-0.042980,-0.037077,-0.034269,-0.031412,-0.006170,-0.011738,-0.051788,-0.046078,430
7498,10,Даниил Медведев во время четвертого сета Как ж...,-0.013558,0.128117,0.361276,0.347715,-0.195470,0.513713,-0.136279,0.456701,...,0.004140,0.003558,-0.070606,-0.057361,0.023872,0.032804,-0.034576,-0.001304,-0.016874,100


In [34]:
import numpy as np

arr1 = np.array([[1],
                   [2],
                   [3]])

arr2 = np.array([[4],
                   [5],
                   [6]])

arr3 = np.array([[7],
                   [8],
                   [9]])

print(arr3)

result = np.hstack((arr1, arr2, arr3))
print(result)

[[7]
 [8]
 [9]]
[[1 4 7]
 [2 5 8]
 [3 6 9]]
